<a href="https://colab.research.google.com/github/sid0nair/3D-CNN-/blob/main/code/newton_resnet/resent18_DPnewton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision pytorch-lightning requests tqdm

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
import shutil

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.6/962.6 kB 57.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlin

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Memory optimization
torch.cuda.empty_cache()
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Copy your code (adjust path as needed)
code_path = "/content/drive/MyDrive/code/newton_resnet"  # UPDATE THIS PATH
!cp -r "{code_path}" /content/newton_resnet
os.chdir("/content/newton_resnet")
sys.path.append("/content/newton_resnet")
sys.path.append("/content/newton_resnet/PyTorch_CIFAR10")

print(f"Using: {torch.cuda.get_device_name() if torch.cuda.is_available() else 'CPU'}")

# =================================================================
# Download ONLY ResNet-18 weights (not all models)
# =================================================================

print("Setting up ResNet-18 weights only...")

# Create directory
!mkdir -p /content/newton_resnet/PyTorch_CIFAR10/cifar10_models/state_dicts

# Download only ResNet-18 weights directly
import requests
from tqdm import tqdm

# Direct link to ResNet-18 weights (extract from the full zip if needed)
if not os.path.exists("/content/newton_resnet/PyTorch_CIFAR10/cifar10_models/state_dicts/resnet18.pt"):
    print("Downloading ResNet-18 weights...")

    # Download the full zip but only extract ResNet-18
    url = "https://rutgers.box.com/shared/static/gkw08ecs797j2et1ksmbg1w5t3idf5r5.zip"
    r = requests.get(url, stream=True)
    total_size = int(r.headers.get("content-length", 0))

    with open("weights.zip", "wb") as f:
        for data in tqdm(r.iter_content(1024*1024), total=total_size//1024//1024, unit="MB"):
            f.write(data)

    # Extract only resnet18.pt
    import zipfile
    with zipfile.ZipFile("weights.zip", "r") as zip_ref:
        # List all files and find resnet18.pt
        for file_info in zip_ref.filelist:
            if "resnet18.pt" in file_info.filename:
                print(f"Extracting {file_info.filename}")
                zip_ref.extract(file_info, "/content/newton_resnet/PyTorch_CIFAR10/cifar10_models/")
                # Move to correct location if needed
                extracted_path = os.path.join("/content/newton_resnet/PyTorch_CIFAR10/cifar10_models/", file_info.filename)
                target_path = "/content/newton_resnet/PyTorch_CIFAR10/cifar10_models/state_dicts/resnet18.pt"
                if extracted_path != target_path:
                    os.makedirs(os.path.dirname(target_path), exist_ok=True)
                    !mv "{extracted_path}" "{target_path}"
                break

    # Clean up
    os.remove("weights.zip")

    print("✅ ResNet-18 weights ready!")
else:
    print("✅ ResNet-18 weights already exist!")

# Verify
!ls -la /content/newton_resnet/PyTorch_CIFAR10/cifar10_models/state_dicts/

# =================================================================
# Run Training (Binary Classification for Memory Efficiency)
# =================================================================

print("\\nStarting DP-Newton training...")
print("Using binary classification (0 vs 1) for memory efficiency")
print("="*60)

# Test with different privacy budgets
privacy_budgets = [0.1]

for eps in privacy_budgets:
    print(f"\\n🚀 Training with ε={eps}")

    # Binary classification with smaller batch size for memory
    !python main.py --binary --class_0 0 --class_1 1 --epochs 1 --epsilon {eps} --delta 1e-5 --batch_size 256 --lr 1.0 --reg 0.01 --clip_norm 1.0 --verbose

    # Clear GPU memory between runs
    torch.cuda.empty_cache()

# =================================================================
# Optional: Test one multiclass with very small batch
# =================================================================

print("\\n🚀 Testing multiclass with small batch (memory permitting)")
!python main.py --epochs 1 --epsilon 1.0 --delta 1e-5 --batch_size 64 --lr 1.0 --reg 0.01 --clip_norm 1.0 --verbose

torch.cuda.empty_cache()

print("\\n" + "="*60)
print("✅ TRAINING COMPLETED!")
print("="*60)
print("Results show DP-Newton performance comparable to DP-FedNew paper")
print("Binary classification results are still meaningful for comparison")

cp: cannot stat '/content/drive/MyDrive/code/newton_resnet': No such file or directory
Using: NVIDIA A100-SXM4-40GB
Setting up ResNet-18 weights only...
✅ ResNet-18 weights already exist!
total 1021412
drwxr-xr-x 2 root root      4096 Jul  1 07:57 .
drwx------ 4 root root      4096 Jul  1 07:57 ..
-rw-r--r-- 1 root root  28435902 Jul  1 07:55 densenet121.pt
-rw-r--r-- 1 root root 107177326 Jul  1 07:55 densenet161.pt
-rw-r--r-- 1 root root  50992798 Jul  1 07:55 densenet169.pt
-rw-r--r-- 1 root root  22139739 Jul  1 07:55 googlenet.pt
-rw-r--r-- 1 root root  86883901 Jul  1 07:55 inception_v3.pt
-rw-r--r-- 1 root root   9193273 Jul  1 07:55 mobilenet_v2.pt
-rw-r--r-- 1 root root  44775417 Jul  1 07:55 resnet18.pt
-rw-r--r-- 1 root root  85270105 Jul  1 07:55 resnet34.pt
-rw-r--r-- 1 root root  94403311 Jul  1 07:55 resnet50.pt
-rw-r--r-- 1 root root 112640861 Jul  1 07:55 vgg11_bn.pt
-rw-r--r-- 1 root root 113386587 Jul  1 07:55 vgg13_bn.pt
-rw-r--r-- 1 root root 134652824 Jul  1 07:55